# 用朴素贝耶斯算法对评论进行分类

In [16]:
# 导入所需的包

import pandas as pd
import jieba
from gensim import corpora,models,matutils

import gensim



In [17]:
# 读取数据，数据标签的雅元处理

data_df  = pd.read_excel("./data/comment.xlsx")

# 标签哑处理

grade_dict = {"力荐":5, "推荐":4, "还行":3, "较差":2, "很差":1}
data_df["grade_new"] = data_df["grade"].map(grade_dict)


print(data_df)

     Unnamed: 0                                            comment grade  \
0             0                          2颗星给拍摄场面、给志愿军战士们，其他没法更多了。    较差   
1             1  非常合格的献礼片，3个小时略长但是看点满满。节奏情感战争场面都处理的恰到好处。虽然是三个大导...    力荐   
2             2                           “《长津湖》最好的彩蛋，便是今日繁荣昌盛的祖国”    力荐   
3             3                          我觉得下次别设置这种任性小孩形象了，战争不是儿戏。    还行   
4             4  看了点映，第七穿插连的故事有血有肉有笑有泪，很是动人。其中恢宏大场面相当有气势，虽然制作时间...    力荐   
..          ...                                                ...   ...   
174         175  三个小时没看够，最后戛然而止，期待下部春节上映。三个导演融合的还行，没有太大的割裂感，战争场...    力荐   
175         176                    向英雄的致敬是无法用打分来衡量的，但是电影的呈现就那样那样吧。    还行   
176         177                    我失望的是这么恢弘壮美的历史被拍成了一部中规中矩的普通战争片。    较差   
177         178                                         一段意味深长的历史！    力荐   
178         179  还是很适合普通观众进电影院去看的，时间虽然长，但节奏控制的比较好，剧情也有递进，也并不复杂，...    力荐   

     grade_new  
0            2  
1            5  
2            5  
3            3  
4 

## 文本的预处理

### 分词、过滤停用词

In [18]:
word_pre = []

# 加载停用词库

with open ("./data/哈工大停用词表.txt") as f:
    stop_word_list = [i.strip() for i in f.readlines()]

# 定义分词、过滤停用词的函数

def division_stop_word(text):

    division_word = jieba.lcut(text)

    result_list = []

    for i in division_word:
        if i>=u"\u4e00" and i<= u"\u9fa5":
            if i not in stop_word_list:
                result_list.append(i)

    word_pre.append(result_list)

data_df["comment"].apply(lambda x: division_stop_word(x))


0      None
1      None
2      None
3      None
4      None
       ... 
174    None
175    None
176    None
177    None
178    None
Name: comment, Length: 179, dtype: object

### 生成文本向量矩阵、词袋、语料

In [19]:

# 生成语料

dictionary  = corpora.Dictionary(word_pre)

# 生成词袋

word_bow = [dictionary.doc2bow(text) for text in word_pre]

# 通过词袋训练模型

tfidf = models.TfidfModel(word_bow)

# 得出词的权重

word_weight = tfidf[word_bow]



# 文本的特征数量组成向量矩阵
word_matrix = matutils.corpus2dense(word_weight,num_terms=len(dictionary.token2id.keys())).T




In [20]:
# 导入算法、训练算法

from sklearn.naive_bayes import GaussianNB  # 高斯分布

from sklearn.naive_bayes import MultinomialNB # 多项式分布

from sklearn.naive_bayes import BernoulliNB # 二项分布


gsnb = GaussianNB()

gsnb.fit(word_matrix,data_df["grade_new"])

gsnb.score(word_matrix,data_df["grade_new"])

0.9888268156424581